<a href="https://colab.research.google.com/github/ShasmaAfs/RAG-School-Certificate/blob/main/RAG_school_certificate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install numpy==1.26.4 --force-reinstall --no-cache-dir


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 136.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have nump

In [1]:
pip install chromadb==0.4.22


In [2]:
pip install sentence-transformers pdfplumber pytesseract python-dotenv transformers accelerate safetensors


In [4]:
from google.colab import files

uploaded = files.upload()
file_name = list(uploaded.keys())[0]

print("Uploaded:", file_name)


Saving school.png to school.png
Uploaded: school.png


In [5]:
import pdfplumber
from PIL import Image
import pytesseract

extracted_text = ""

if file_name.endswith(".pdf"):
    with pdfplumber.open(file_name) as pdf:
        for page in pdf.pages:
            extracted_text += page.extract_text() + "\n"
else:
    img = Image.open(file_name)
    extracted_text = pytesseract.image_to_string(img)

print("Extracted Text:\n", extracted_text)

Extracted Text:
 IJIULTIVWVUVL

CERTIFICATE

Student Name:
Siyasath Shasma
School:

Mahmud Ladies College
Year:

2019
Subjects: Chemistry, Physics, Maths

Issued by:
Ministry of Education



In [6]:
import chromadb
from chromadb.utils import embedding_functions

embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

chroma_client = chromadb.Client()
collection = chroma_client.create_collection(
    name="School_Certificate_Rag",
    embedding_function=embedding_function
)

collection.add(
    documents=[extracted_text],
    ids=["school_certificate"]
)

print("Vector DB ready!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Vector DB ready!


In [7]:
from transformers import pipeline

qa_model = pipeline("question-answering", model="deepset/roberta-base-squad2")


def rag_query(question):
    results = collection.query(
        query_texts=[question],
        n_results=1
    )

    context = results["documents"][0][0]

    answer = qa_model({
        'question': question,
        'context': context
    })

    return answer['answer']


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cpu


In [9]:
print(rag_query("What is the student's name?"))

Siyasath Shasma


In [10]:
print(rag_query("What is the school name?"))

Mahmud Ladies College


In [11]:
print(rag_query("What is the year?"))

2019


In [12]:
print(rag_query("What are the subjects?"))

Chemistry, Physics, Maths


In [13]:
print(rag_query("Who issued it?"))


Ministry of Education

